In [1]:
# https://ieeexplore.ieee.org/abstract/document/8203676
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df=pd.read_csv("01_District_wise_crimes_committed_IPC_2001_2012.csv")

In [3]:
df.head(5)
print(df.shape)

(9017, 33)


In [4]:
year=df["YEAR"].unique()

In [5]:
year

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012], dtype=int64)

In [6]:
total_crimes={}
for i in year:
    df1=df[(df["YEAR"]==i) & ((df["DISTRICT"]=="TOTAL") |( df["DISTRICT"]=="DELHI UT TOTAL")) ]
    #print(df1.head(5))
    total_crime=df1["TOTAL IPC CRIMES"].sum()
    total_crimes[i]=total_crime
#print(total_crimes)
    
    
    

In [7]:
t1 = pd.DataFrame(list(total_crimes.items()),columns = ['year','crimes']) 

In [8]:
import seaborn as sns

In [9]:
total_crimes={}
state=df["STATE/UT"].unique()
for i in state:
    df1=df[(df["STATE/UT"]==i) & (df["DISTRICT"]=="TOTAL") |( df["DISTRICT"]=="DELHI UT TOTAL")]
    #print(df1.head(5))
    total_crime=df1["TOTAL IPC CRIMES"].sum()
    total_crimes[i]=total_crime
#print(total_crimes)
    
    
    

In [10]:

t2 = pd.DataFrame(list(total_crimes.items()),columns = ['STATES','crimes'])  
print(t2.shape)
print(t2)

(35, 2)
               STATES   crimes
0      ANDHRA PRADESH  2652155
1   ARUNACHAL PRADESH   660826
2               ASSAM  1230938
3               BIHAR  1979467
4        CHHATTISGARH  1194201
5                 GOA   665225
6             GUJARAT  2018949
7             HARYANA  1228477
8    HIMACHAL PRADESH   788122
9     JAMMU & KASHMIR   892329
10          JHARKHAND  1055525
11          KARNATAKA  2114237
12             KERALA  2070633
13     MADHYA PRADESH  3046944
14        MAHARASHTRA  2906610
15            MANIPUR   668246
16          MEGHALAYA   658423
17            MIZORAM   659320
18           NAGALAND   646307
19             ODISHA  1281120
20             PUNJAB  1017305
21          RAJASTHAN  2489090
22             SIKKIM   640174
23         TAMIL NADU  2693350
24            TRIPURA   685908
25      UTTAR PRADESH  2491248
26        UTTARAKHAND   736378
27        WEST BENGAL  1752478
28      A & N ISLANDS   642276
29         CHANDIGARH   673981
30       D & N HAVELI   637825


In [11]:
total_women_crimes={}
state=df["STATE/UT"].unique()
for i in state:
    df1=df[(df["STATE/UT"]==i) & (df["DISTRICT"]=="TOTAL") |( df["DISTRICT"]=="DELHI UT TOTAL")]
    #print(df1.head(5))
    total_crime=0
    total_crime=total_crime+df1["ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY"].sum()
    total_crime=total_crime+df1["RAPE"].sum()
    total_crime=total_crime+df1["KIDNAPPING AND ABDUCTION OF WOMEN AND GIRLS"].sum()
    total_crime=total_crime+df1["DOWRY DEATHS"].sum()
    total_women_crimes[i]=total_crime

    
sc = pd.DataFrame(list(total_women_crimes.items()),columns = ['STATES','crimes'])
# print(sc.shape)
sc['tcrime']=t2['crimes']
sc['crimes']=sc['crimes']/sc['tcrime']

women_rank={}
top=sc.nlargest(35, ['crimes']) 
# print(top)
l=np.array(top['STATES'])
m=[]
for i in range(34,-1,-1):
    women_rank[l[i]]=36-(i+1)
# print(women_rank)

l=[]

for i in t2['STATES']:
    l.append(women_rank[i])

t2['women safety index']=np.array(l)   

t3=t2.nlargest(5, ['crimes']) 

In [12]:
# pv = t2.pivot_table(values='women crime rank',index='STATES',columns='crimes')
# sns.heatmap(pv)

In [13]:
import json

In [14]:

india_states=json.load(open("states_india.geojson",'r'))


In [15]:
#india_states['features'][1]

In [16]:
print(india_states['features'][0].keys())
print(india_states['features'][0]['properties'])

dict_keys(['type', 'geometry', 'properties'])
{'cartodb_id': 1, 'state_code': 0, 'st_nm': 'Telangana'}


In [17]:
state_id_map={}
for feature in india_states['features']:
    feature['id']=feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']]=feature['id']

In [18]:
print(india_states['features'][0].keys())

dict_keys(['type', 'geometry', 'properties', 'id'])


In [19]:
#state_id_map

In [20]:
#print(t2['STATES'])

state_id_map['Arunachal Pradesh']=state_id_map["Arunanchal Pradesh"]
state_id_map.pop('Arunanchal Pradesh')
state_id_map = dict((k.upper(), v) for k, v in state_id_map.items())
j=[i for i in state_id_map.keys()]
for i in j:
    if  i not  in t2['STATES'].values:
        #print("rter")
        state_id_map.pop(i.upper())

In [21]:
#state_id_map

In [22]:
import plotly.express as px

In [23]:
import plotly.io as pio
pio.renderers.default = "browser"

In [24]:
for i in t2.STATES.values:
    if i not in state_id_map.keys():
       t2= t2[t2.STATES!=i]

In [25]:

t2['id']=t2['STATES'].apply(lambda x: state_id_map[x])

In [26]:
#fig=px.choropleth(t2,locations='id', geojson=india_states ,color="crimes" ,scope="asia",hover_name="STATES") 
#fig.update_geos(fitbounds="locations",visible=False)

In [27]:
#fig.show()

In [28]:
fig=px.choropleth_mapbox(t2,locations='id', geojson=india_states ,color="crimes" 
                         ,hover_name="STATES",hover_data=["women safety index"],mapbox_style="carto-positron",center={'lat':24,'lon':78},
                        zoom=3,opacity=0.5) 
fig.show()
